In [2]:
# Instantiate the Cohere client

import cohere
import os
import pprint

COHERE_API_KEY = 'tBZEDWJ0DN7QIWxNrz525ZiW8rnU2JMT2Zvv5pNv'
co = cohere.Client(api_key=COHERE_API_KEY)
pp = pprint.PrettyPrinter(indent=4)

from tools import tools, invoke_tool


In [3]:
PREAMBLE = """
## Context
You are a rube goldberg machine. You make the most craziest plans possible. 

You are given a bunch of tools to help you generate responses. 
You may need to use multiple tools in parallel or sequentially to complete your task. Think step-by-step.
Make sure to use the most number of tools. Use path to complete the task.

## Style Guide
You should answer in gen-Z voice.

Example:
User: I want a healty plan for today.
You: Okay, let's go to this nearest pizza restaurant, order a pepperoni pizza and then we'll go to a long marathon run.
"""

In [16]:
goal = "Make me a bunch of money!"
message = "Develop a step by step plan to help the user achieve the goal: " + goal + ". Include acceptance criteria for each step that guarantees good craic is had by all. Your answer must be provided as a list of steps with the following structure: STEP: step number; ACTION: step to be carried out; ACCEPTANCE: criteria that must be met for the step to be considered complete.\n. Do not use markdown, only plaintext. Each step, action and acceptance criteria must be in the same line and not use newline or semicolons"

In [17]:
res = co.chat(
            model="command-r",
            preamble=PREAMBLE,
            message=message,
            force_single_step=False,
            tools=tools
        )
        
steps = [{
    "text": res.text,
    "tool_calls": [{"name": call.name, "parameters": call.parameters} for call in (res.tool_calls or [])]
}]

In [6]:
print(steps)


[{'text': 'I will search for ways to make money and present the user with a plan with the most absurd steps possible.', 'tool_calls': [{'name': 'web_search', 'parameters': {'query': 'ways to make money'}}]}]


In [18]:
while res.tool_calls:
    tool_results = []
    for call in res.tool_calls:
        tool_result = invoke_tool(call)
        tool_results.append({
            "call": {"name": call.name, "parameters": call.parameters},
            "outputs": tool_result
        })
    
    res = co.chat(
        model="command-r",
        preamble=PREAMBLE,
        chat_history=res.chat_history,
        message="",
        force_single_step=False,
        tools=tools,
        tool_results=tool_results,
    )
    
    steps.append({
        "text": res.text,
        "tool_calls": [{"name": call.name, "parameters": call.parameters} for call in (res.tool_calls or [])],
        "tool_results": tool_results
    })

last_step = steps[-1]
pp.pprint(res.text)

Searching the web for ways to make money
Web search results for ways to make money: [{'title': '16 Ways to Make Money From Home in 2024 - Forbes', 'href': 'https://www.forbes.com/advisor/business/make-money-from-home/', 'body': 'Learn how to earn extra income from home with flexible and remote opportunities, such as room rentals, online tutoring, freelance writing and more. Compare the benefits, requirements and pay of different work-from-home options.'}, {'title': '25 Ways to Make Money Online and Offline - NerdWallet', 'href': 'https://www.nerdwallet.com/article/finance/how-to-make-money', 'body': 'Find out how to make money with a side gig and how long it will take to see the extra income. Learn about freelance work, website testing, AI tools, surveys and more.'}, {'title': '75 Ways To Make Money: Online, Offline & From Home - Millennial Money', 'href': 'https://millennialmoney.com/ways-to-make-money/', 'body': 'Learn how to earn extra cash with various side hustles, from micro-task

In [19]:
import re
parsed = res.text.split("\n")
parsed = list(filter(lambda x: x != "", parsed))
parsed = list(filter(lambda x: re.match(r"^\d+.", x), parsed))
print(parsed)


['1. STEP: 1; ACTION: Search for ways to make money from home; ACCEPTANCE: Methods found and confirmed to make money.', '2. STEP: 2; ACTION: Rent out a room; ACCEPTANCE: Room listed on at least 3 apps/websites.', '3. STEP: 3; ACTION: Become an online tutor; ACCEPTANCE: Registered with 2 tutoring websites. ', '4. STEP: 4; ACTION: Start freelance writing; ACCEPTANCE: Registered with 2 freelance writing websites.', '5. STEP: 5; ACTION: Explore freelance work; ACCEPTANCE: Applied for 3 freelance jobs.', '6. STEP: 6; ACTION: Do website testing; ACCEPTANCE: Tested 2 websites. ', '7. STEP: 7; ACTION: Look into AI tools; ACCEPTANCE: Downloaded 2 AI tools.', '8. STEP: 8; ACTION: Complete surveys; ACCEPTANCE: Signed up for 3 survey websites.', '9. STEP: 9; ACTION: Search for micro-tasks; ACCEPTANCE: Found 3 websites with micro-tasks.', '10. STEP: 10; ACTION: Become a pet sitter; ACCEPTANCE: Registered with 2 pet sitting websites.', '11. STEP: 11; ACTION: Start a blog; ACCEPTANCE: Blog live with 

In [20]:
step = parsed[0]
print(step.split(";"))
_, action, acceptance = step.split(";")
print(step)

['1. STEP: 1', ' ACTION: Search for ways to make money from home', ' ACCEPTANCE: Methods found and confirmed to make money.']
1. STEP: 1; ACTION: Search for ways to make money from home; ACCEPTANCE: Methods found and confirmed to make money.
